# UCN LD2 Calculations

J. Martin, K. Augusto

May 25/20

Modeling a HEX as a single straight tube with fluid flowing down the middle using https://uspas.fnal.gov/materials/10MIT/Lecture_3.2.pdf as a reference. A fixed Reynolds number of 10,000 is used. 

In [4]:
#!/usr/bin/python3

from math import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import numpy as np
from scipy import interpolate
from scipy.integrate import odeint

The flow perimeter is found from:

\begin{equation}
P = \frac{4\dot{m}}{Re\mu}
\end{equation}

In [5]:
mdot=0.004 # kg/s
mu=3.5e-5 # Pa*s
Re=10000.
p=4*mdot/(Re*mu) # m
print('Flow perimeter %f m'%p)
diameter=p/pi
print('Diameter of tube %f m'%diameter)

rho=163.0 # kg/m^3

Cp=6565.4 # J/(kg*K)

Flow perimeter 0.045714 m
Diameter of tube 0.014551 m


The Colburn Factor is found from:

\begin{equation}
jH = 0.023Re^{-0.2}B1
\end{equation}

The Nusselt Number is found from:

\begin{equation}
Nu = jHRePr^{1/3}
\end{equation}

In [6]:
Pr=2.209513 # taken from d2-hex-drawing.py
B1=1.174*((3.7e-5)/(3.68e-5))**(0.14) #viscosity taken from cams sheets
print('This is B1 %f.' %B1)
jh=0.023*Re**(-0.2)*B1
Nu=jh*Re*Pr**(1./3.)
print('According to correlations, Nu=%f'%Nu)

This is B1 1.174891.
According to correlations, Nu=55.781604


The heat transfer coefficient is found from:

\begin{equation}
hc = \frac{Nu kt}{D}
\end{equation}

In [7]:
kt=0.104 # W/(m*K) 
Ntu=3.0
hc=Nu*kt/diameter
print('The heat transfer coefficient is %f W/m^2-K'%hc)

The heat transfer coefficient is 398.678003 W/m^2-K


The length of the tube needed is found from:

\begin{equation}
L = \frac{Ntu\dot{m}C_p}{Nu kt\pi}
\end{equation}

The friction factor is found from:

\begin{equation}
f = 0.316Re^{-0.25}
\end{equation}

In [8]:
L=Ntu/(Nu*kt*pi/(mdot*Cp))
print('Tube length needed for sufficient heat transfer %f m'%L)

f=0.316*Re**(-0.25)
print('The turbulent friction factor is %f.' %f)

Tube length needed for sufficient heat transfer 4.322831 m
The turbulent friction factor is 0.031600.


The pressure drop is found from:

\begin{equation}
dp = \frac{8f\dot{m}^2L}{\rho \pi^2 D^5}
\end{equation}

In [9]:
dp=(8*f*mdot**2/(rho*pi**2))*L/diameter**5 # tube
print('The pressure drop is %f Pa'%dp)

The pressure drop is 16.659716 Pa


In [10]:
dcoil=4.*0.0254 # m diameter of coiled tube
turns=L/(pi*dcoil)
print('Coiling around a Cu rod of diameter %f m would require %f turns'%(dcoil,turns))
print('This Cu rod would need to be at least %f m long (for zero Cu wall thickness)'%(turns*diameter))

Coiling around a Cu rod of diameter 0.101600 m would require 13.543305 turns
This Cu rod would need to be at least 0.197073 m long (for zero Cu wall thickness)


In the above calculations, Reynolds number was fixed at 10,000 and the diameter at 0.014551 m, to find what the pressure drop would be. Now, the pressure drop is fixed at 10 Pa and the Ntu is fixed at 3, to determine what the diameter of the tube, the Reynolds number, and the heat transfer coefficient would be at 10 Pa. 


Finding the optimal pipe diameter for a pressure drop of 10 Pa by:

\begin{equation}
D = \frac{8f\dot{m}^2L}{(dp\rho\pi^2)^{1/5}}
\end{equation}

In [11]:
dp2=10 #Pa
mdot=0.004 # kg/s
mu=3.5e-5 # Pa*s
Ntu=3.0
f=0.03 # following along in the example, have to guess an f

D=((8*f*mdot**2*L)/(rho*pi**2*dp2))**(1/5)

print('The diameter is %f m'%D)

The diameter is 0.015949 m


The Reynolds Number is found from:

\begin{equation}
Re = \frac{4\dot{m}}{ \pi D \mu}
\end{equation}

The Nusselt Number is found from:

\begin{equation}
Nu = jH RePr^{1/3}
\end{equation}

In [12]:
Re2=(4*mdot)/(pi*D*mu)
print('This is Re based on optimal diameter %f' %Re2)

Pr2=(mu*Cp)/(kt) # yes still dimensionless
jh2=0.023*Re2**(-0.2)*B1
Nu2=jh2*Re2*Pr2**(1./3.)
print('According to correlations, Nu=%f'%Nu2)

This is Re based on optimal diameter 9123.881703
According to correlations, Nu=51.836360


The length of the tube needed is found from:

\begin{equation}
L = \frac{Ntu\dot{m}C_p}{Nu kt\pi}
\end{equation}

In [13]:
L2=Ntu/(Nu2*kt*pi/(mdot*Cp))
print('Tube length needed for sufficient heat transfer %f m'%L)

turns2=L2/(pi*D)
print('Coiling around a HEX of diameter %f m would require %f turns'%(D,turns2))
print('This Cu rod would need to be at least %f m long (for zero Cu wall thickness)'%(turns2*D))

Tube length needed for sufficient heat transfer 4.322831 m
Coiling around a HEX of diameter 0.015949 m would require 92.843700 turns
This Cu rod would need to be at least 1.480727 m long (for zero Cu wall thickness)


The heat transfer coefficient is found from:

\begin{equation}
hc = \frac{Nukt}{D}
\end{equation}

The Nusselt number from the example is found from:

\begin{equation}
Nu = 0.023Re^{0.8}Pr^{0.3}
\end{equation}

In [14]:
hc2=Nu2*kt/D
print('The heat transfer coefficient is %f W/m^2-K'%hc2)

Nu3=0.023*Re2**(0.8)*Pr2**(0.3)
print('From example Nu=%f'%Nu3)

The heat transfer coefficient is 338.022348 W/m^2-K
From example Nu=42.969502


The heat transfer per temperature is found from:

\begin{equation}
Q = hc\pi DL
\end{equation}

In [15]:
QL=hc2*pi*D*L
print('The heat transfer per temp is %f W/K' %QL)

The heat transfer per temp is 73.212617 W/K
